# AI Task 1 – Industrial Machine Failure Prediction

## Problem Statement
The objective of this task is to build a simple and reliable machine learning model to predict whether an industrial machine operation will fail or not.

Each row in the dataset represents one machine operation, along with sensor measurements, machine type, and a failure indicator.  
The focus of this task is correctness, clear reasoning, and proper evaluation rather than complex modeling.


In [18]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


Load Dataset

In [19]:
df = pd.read_csv("Dataset.csv")
df.head()


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [20]:
df.columns


Index(['UDI', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Target', 'Failure Type'],
      dtype='object')

In [21]:
df.columns = df.columns.str.strip()


In [22]:
df.columns


Index(['UDI', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Target', 'Failure Type'],
      dtype='object')

## Dataset Overview
We begin by inspecting the structure, size, and data types to understand what kind of preprocessing is required.


In [23]:
df.shape
df.info()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      10000 non-null  int64  
 1   Product ID               10000 non-null  object 
 2   Type                     10000 non-null  object 
 3   Air temperature [K]      10000 non-null  float64
 4   Process temperature [K]  10000 non-null  float64
 5   Rotational speed [rpm]   10000 non-null  int64  
 6   Torque [Nm]              10000 non-null  float64
 7   Tool wear [min]          10000 non-null  int64  
 8   Target                   10000 non-null  int64  
 9   Failure Type             10000 non-null  object 
dtypes: float64(3), int64(4), object(3)
memory usage: 781.4+ KB


,0
UDI,0
Product ID,0
Type,0
Air temperature [K],0
Process temperature [K],0
Rotational speed [rpm],0
Torque [Nm],0
Tool wear [min],0
Target,0
Failure Type,0


## Understanding the Target Variable
The target column indicates whether an operation resulted in a machine failure.
Since this is a real-world industrial dataset, class imbalance is expected.


In [24]:
df['Target'].value_counts()


,count
Target,
0,9661
1,339


## Feature Understanding and Selection

- ID columns are identifiers and do not contribute predictive value.
- The text-based failure description directly reflects the target and may cause data leakage.
- Numerical sensor readings are the main predictive features.
- Machine type is a categorical feature and needs encoding.


In [25]:
X = df.drop(columns=['Target'])
y = df['Target']


In [26]:
X = X.drop(columns=[
    'UDI',          # Unique ID
    'Product ID',   # Product identifier
    'Failure Type'  # Textual failure description
])


## Feature Categorization
We explicitly separate categorical and numerical features to apply appropriate preprocessing.


In [27]:
categorical_cols = ['Type']

numerical_cols = [
    'Air temperature [K]',
    'Process temperature [K]',
    'Rotational speed [rpm]',
    'Torque [Nm]',
    'Tool wear [min]'
]


## Preprocessing Pipeline

- Categorical features are one-hot encoded.
- Numerical features are standardized.
- A ColumnTransformer ensures preprocessing is applied consistently.


In [28]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', StandardScaler(), numerical_cols)
    ]
)


## Train-Test Split

A stratified split is used to preserve the failure distribution in both training and test sets.


In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


## Model Selection

Logistic Regression is chosen because:
- It is well-suited for binary classification
- It is interpretable and simple
- It aligns with the task constraint of avoiding complex models


In [30]:
model = LogisticRegression(max_iter=1000)


In [31]:
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', model)
])


## Model Training
The model is trained only on the training dataset to avoid data leakage.


In [32]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Type']),
                                                 ('num', StandardScaler(),
                                                  ['Air temperature [K]',
                                                   'Process temperature [K]',
                                                   'Rotational speed [rpm]',
                                                   'Torque [Nm]',
                                                   'Tool wear [min]'])])),
                ('model', LogisticRegression(max_iter=1000))])

## Model Evaluation
Multiple evaluation metrics are used instead of relying solely on accuracy.
This is important because missing a failure can be more costly than a false alarm.


In [33]:
y_pred = pipeline.predict(X_test)


In [34]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9675

Confusion Matrix:
 [[1928    4]
 [  61    7]]

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      1932
           1       0.64      0.10      0.18        68

    accuracy                           0.97      2000
   macro avg       0.80      0.55      0.58      2000
weighted avg       0.96      0.97      0.96      2000



## Results Interpretation

- Accuracy alone can be misleading due to class imbalance.
- Precision and recall provide better insight into failure detection.
- Recall for the failure class indicates how many actual failures are correctly identified.


## Limitations and Future Improvements

If given more time, the following improvements could be explored:
- Feature engineering based on sensor trends
- Cost-sensitive learning to penalize missed failures
- Threshold tuning to improve failure recall
- Cross-validation for more robust evaluation
